In [1]:
import glob
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import os

In [2]:
filepaths = glob.glob("..\\raw_data\\**\\*.jpg", recursive=True)

In [3]:
df = pd.DataFrame(filepaths, columns=["img_id"])
df["img_id"] = df["img_id"].apply(lambda x: x.replace("..\\raw_data\\", ""))
df["plant"] = df["img_id"].apply(lambda x: x.split("\\")[0])
df["condition"] = df["img_id"].apply(lambda x: x.split("\\")[1])
df["plant_condition"] = df["plant"] + "_" + df["condition"]

In [4]:
# Source: https://jdhao.github.io/2017/11/06/resize-image-to-square-with-padding/
def pad_resize(img, desired_size=256):
    old_size = img.shape[:2]
    ratio = float(desired_size) / max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])

    temp = cv2.resize(img, (new_size[1], new_size[0]))

    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]

    top, bottom = delta_h // 2, delta_h - delta_h // 2
    left, right = delta_w // 2, delta_w - delta_w // 2

    new_img = cv2.copyMakeBorder(temp, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    
    return new_img

In [5]:
for _, row in tqdm(df.iterrows(), total=len(df)):
    img = cv2.imread("..\\raw_data\\" + row["img_id"])
    img = pad_resize(img, desired_size=256)

    outpath = "..\\preprocessed_data\\" + row["img_id"]
    outdir = os.path.dirname(outpath)

    if not os.path.isdir(outdir):
        os.makedirs(outdir)

    cv2.imwrite(outpath, img)